In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"harryliu0701","key":"a98edf900862418ec3d14ffae1f570eb"}'}

In [4]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json



kaggle.json


In [5]:
!kaggle datasets download -d drgfreeman/rockpaperscissors

 95% 292M/306M [00:05<00:00, 55.1MB/s]
100% 306M/306M [00:05<00:00, 63.1MB/s]


In [6]:
!unzip -q rockpaperscissors.zip -d Rock-Paper-Scissors/

In [ ]:
!pip install Keras-Applications

In [9]:
!mkdir train/
!mkdir test/
!mkdir train/scissors/
!mkdir train/rock/
!mkdir train/paper/
!mkdir test/scissors/
!mkdir test/rock/
!mkdir test/paper/

In [10]:
!ls Rock-Paper-Scissors/scissors/ | shuf -n 450 | xargs -I {} mv Rock-Paper-Scissors/scissors/{} train/scissors/
!ls Rock-Paper-Scissors/rock/ | shuf -n 450 | xargs -I {} mv Rock-Paper-Scissors/rock/{} train/rock/
!ls Rock-Paper-Scissors/paper/ | shuf -n 450 | xargs -I {} mv Rock-Paper-Scissors/paper/{} train/paper/

In [11]:
!mv Rock-Paper-Scissors/paper/* test/paper/
!mv Rock-Paper-Scissors/rock/* test/rock/
!mv Rock-Paper-Scissors/scissors/* test/scissors/

In [12]:
!mkdir model/

In [3]:
import torch
import os
import cv2
import random
import torch.utils.data as data
import numpy as np
import torchvision
from torchvision.models import ResNet
from pathlib import Path
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import math
import torch.nn as nn
from tqdm import tqdm
import scipy.io
from torch.autograd import Variable

In [4]:
class DataFolder(data.Dataset):
    def __init__(self, root):
        image_set = []
        label_set = []
        
        go_rock = os.walk(root + "rock/")
        for path,d,filelist in go_rock:  
            for filename in filelist:
                if filename.endswith('.png'):
                    file_root = root + "rock/" + filename
                    image_set.append(file_root)
                    label_set.append(0)
        go_paper = os.walk(root + "paper/")
        for path,d,filelist in go_paper: 
            for filename in filelist:
                if filename.endswith('.png'):
                    file_root = root + "paper/" + filename
                    image_set.append(file_root)
                    label_set.append(1)
        go_scissors = os.walk(root + "scissors/")
        for path,d,filelist in go_scissors:  
            for filename in filelist:
                if filename.endswith('.png'):
                    file_root = root + "scissors/" + filename
                    image_set.append(file_root)
                    label_set.append(2)
        
        self.label = label_set
        self.image = image_set
        
    def __getitem__(self, index):
        image_i = cv2.imread(self.image[index], cv2.IMREAD_COLOR)
        image_reshaped= cv2.resize(image_i, (224,224))
        
        img = image_reshaped
        lb = self.label[index]
        return img, lb
        # return image_i, lb

    def __len__(self):
        return len(self.image)

In [5]:
a = DataFolder(root='./train/')
train_loader = torch.utils.data.DataLoader(dataset=a, batch_size=16, shuffle=True)
b = DataFolder(root='./test/')
test_loader = torch.utils.data.DataLoader(dataset=b, batch_size=8, shuffle=True)

In [24]:
from torchvision.models import squeezenet

In [6]:
num_epochs = 50
batch_size = 16
learning_rate = 0.001

In [33]:
model = torchvision.models.googlenet(pretrained=True)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lossFunction = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


In [34]:
from math import floor, ceil
class SpatialPyramidPooling2d(nn.Module):
    def __init__(self, num_level, pool_type='max_pool'):
        super(SpatialPyramidPooling2d, self).__init__()
        self.num_level = num_level
        self.pool_type = pool_type

    def forward(self, x):
        N, C, H, W = x.size()
        for i in range(self.num_level):
            level = i + 1
            kernel_size = (ceil(H / level), ceil(W / level))
            stride = (ceil(H / level), ceil(W / level))
            padding = (floor((kernel_size[0] * level - H + 1) / 2), floor((kernel_size[1] * level - W + 1) / 2))

            if self.pool_type == 'max_pool':
                tensor = (F.max_pool2d(x, kernel_size=kernel_size, stride=stride, padding=padding)).view(N, -1)
            else:
                tensor = (F.avg_pool2d(x, kernel_size=kernel_size, stride=stride, padding=padding)).view(N, -1)

            if i == 0:
                res = tensor
            else:
                res = torch.cat((res, tensor), 1)
        return res
    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'num_level = ' + str(self.num_level) \
            + ', pool_type = ' + str(self.pool_type) + ')'


In [38]:
class SPPNet(nn.Module):
    def __init__(self, num_level=3, pool_type='max_pool'):
        super(SPPNet,self).__init__()
        self.num_level = num_level
        self.pool_type = pool_type
        self.feature = nn.Sequential(nn.Conv2d(3,16,3),\
                                    nn.ReLU(),\
                                    nn.MaxPool2d(2),\
                                    nn.Conv2d(16,32,3),\
                                    nn.ReLU(),\
                                    nn.MaxPool2d(2),\
                                    nn.Conv2d(32,64,3),\
                                    nn.ReLU(),\
                                    nn.MaxPool2d(2),\
                                    nn.Conv2d(64,128,3),\
                                    nn.ReLU(),\
                                    nn.MaxPool2d(2),\
                                    nn.Conv2d(128,256,3),\
                                    nn.ReLU())
        self.num_grid = self._cal_num_grids(num_level)
        self.spp_layer = SpatialPyramidPooling2d(num_level)
        self.linear = nn.Sequential(nn.Linear(self.num_grid * 256, 512),\
                                    nn.Linear(512, 10))
    def _cal_num_grids(self, level):
        count = 0
        for i in range(level):
            count += (i + 1) * (i + 1)
        return count

    def forward(self, x):
        x = self.feature(x)
        x = self.spp_layer(x)
        x = self.linear(x)
        return x

In [39]:
model = SPPNet()
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lossFunction = nn.CrossEntropyLoss()

In [40]:
total_step = len(train_loader)
correct=0
best_accuracy = 0
accuracy = 0
num =0
k=1
j=1

cnt=1
output_loss=[]
output_acc=[]

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):        
        images = images.view(-1,3,224,224)
        images = torch.autograd.Variable(images).float().cuda()
        out = model(images)        
        labels = torch.autograd.Variable(labels).cuda()
        check = sum(torch.max(out.data,1)[1].cpu() == labels.cpu()).numpy().tolist()
        correct += check
        loss = lossFunction(out, labels)
        num += labels.size(0)
        
        # info = {
        #         'loss': loss.data[0],
        #     }
        # for tag, value in info.items():
        #     logger.scalar_summary(tag, value, k)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        k+=1
        
        if (i+1) % 1 == 0:
            output_loss.append([cnt, loss.data.item()])
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, i+1, total_step, loss.data.item()))
            cnt+=1

    accuracy = 100*correct/num
    output_acc.append([cnt, accuracy])
    # info_2 = {
    #             'accuracy': accuracy
    #         }
    # for tag, value in info_2.items():
    #     logger.scalar_summary(tag, value, j)
        
    correct = 0
    num = 0
    j+=1    
        

Epoch [1/50], Step [1/85], Loss: 2.9524
Epoch [1/50], Step [2/85], Loss: 18.3336
Epoch [1/50], Step [3/85], Loss: 10.8069
Epoch [1/50], Step [4/85], Loss: 4.7089
Epoch [1/50], Step [5/85], Loss: 1.7036
Epoch [1/50], Step [6/85], Loss: 1.9720
Epoch [1/50], Step [7/85], Loss: 1.2759
Epoch [1/50], Step [8/85], Loss: 1.2870
Epoch [1/50], Step [9/85], Loss: 1.1254
Epoch [1/50], Step [10/85], Loss: 1.4697
Epoch [1/50], Step [11/85], Loss: 1.1222
Epoch [1/50], Step [12/85], Loss: 1.1625
Epoch [1/50], Step [13/85], Loss: 1.4103
Epoch [1/50], Step [14/85], Loss: 1.2508
Epoch [1/50], Step [15/85], Loss: 1.0513
Epoch [1/50], Step [16/85], Loss: 1.1454
Epoch [1/50], Step [17/85], Loss: 1.3464
Epoch [1/50], Step [18/85], Loss: 1.1688
Epoch [1/50], Step [19/85], Loss: 1.0678
Epoch [1/50], Step [20/85], Loss: 1.3979
Epoch [1/50], Step [21/85], Loss: 1.3394
Epoch [1/50], Step [22/85], Loss: 1.1934
Epoch [1/50], Step [23/85], Loss: 1.0580
Epoch [1/50], Step [24/85], Loss: 1.2636
Epoch [1/50], Step [25/

In [41]:
rows_loss =  output_loss
np.savetxt("loss.csv",  
           rows_loss, 
           delimiter =", ",  
           fmt ='% s') 

rows_acc =  output_acc
np.savetxt("acc.csv",  
           rows_acc, 
           delimiter =", ",  
           fmt ='% s') 

In [42]:
correct = 0
total = 0
for images,labels in test_loader:
    images = images.view(-1, 3, 224, 224)
    test = Variable(images).float().cuda()
    out = model(test)
    _,predicted = torch.max(out.data,1)
    total += labels.size(0)
    i = 0
    while (i < len(labels)):
        if (predicted[i].cpu().numpy() == labels[i].numpy()):
            correct += 1
        i += 1
print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))

Accuracy of the network on the test images: 97.13603818615752 %


In [43]:
torch.save(model, './model/best_model_sppcnn.pth')

In [44]:
!ls ./model/ -l best_model_sppcnn.pth

ls: cannot access 'best_model_sppcnn.pth': No such file or directory
./model/:
total 8736
-rw-r--r-- 1 root root 8941749 Nov 21 08:50 best_model_sppcnn.pth
